In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

i1 = np.array([
    [0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0],
    [0,0,0,0,1,0,0],
    [0,0,0,1,0,0,0],
    [0,0,1,0,0,0,0],
    [0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0]
])

i2 = np.array([
    [0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0],
    [0,0,1,0,0,0,0],
    [0,0,0,1,0,0,0],
    [0,0,0,0,1,0,0],
    [0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0]
])

c1 = np.array([
    [-1,-1,0],
    [-1,0,1],
    [0,1,1]
])

c2 = np.array([
    [0,1,1],
    [-1,0,1],
    [-1,-1,0]
])

#%%Módulo de convolucion
def convolution(image,kernel):
    i_width, i_height = image.shape[0], image.shape[1]
    k_width, k_height = kernel.shape[0], kernel.shape[1]
    #filtered = np.zeros_like(image)
    #filtered = np.zeros_like(kernel)
    filtered = np.zeros((i_width-k_width+1,i_height-k_height+1))
    #print(filtered)
    for y in range(i_height):
        #print(y)
        if y + k_height <= i_height:
            for x in range(i_width):
                if x + k_width <= i_width:
                    weighted_pixel_sum = 0
                    #print(range(-(k_height // 2), k_height - 1))
                    y_displ=0
                    for ky in range(-(k_height // 2), k_height - 1):
                        #print("ky: ",ky)
                        x_displ=0
                        for kx in range(-(k_width // 2), k_width - 1):
                            pixel = image[y+y_displ,x+x_displ]
                            #print("pixel: ",pixel)
                            weight = kernel[ky + (k_height // 2), kx + (k_width // 2)]
                            #print("weight: ",weight)
                            weighted_pixel_sum += pixel * weight
                            x_displ +=1
                        y_displ+=1
                    filtered[y,x] = max(0,weighted_pixel_sum)
    return filtered

i1_conv1 = convolution(i1,c1)
i1_conv2 = convolution(i1,c2)
i2_conv1 = convolution(i2,c1)
i2_conv2 = convolution(i2,c2)
#print(i1_conv1)

#%% Etapa de reducción de datos
def max_pooling(conv,window):
    #print(window)
    i_width, i_height = conv.shape[0], conv.shape[1]
    w_width, w_height = window.shape[0], window.shape[1]
    reduction = np.zeros((i_width-w_width+1,i_height-w_height+1))
    for y in range(i_height):
        #print(y)
        if y + w_height <= i_height:
            for x in range(i_width):
                if x + w_width <= i_width:
                    max_val = 0
                    #print(range(-(k_height // 2), k_height - 1))
                    y_displ=0
                    for ky in range(-(w_height // 2), w_height - 1):
                        #print("ky: ",ky)
                        x_displ=0
                        for kx in range(-(w_width // 2), w_width - 1):
                            pixel = conv[y+y_displ,x+x_displ]
                            #print("weight: ",weight)
                            if pixel > max_val:
                                max_val = pixel
                        y_displ+=1
                    reduction[y,x] = max_val
    return reduction

#%%Etapa deformación del vector descriptor
def gen_desc_vect(w1,w2):
    descript_vec1 = np.reshape(w1,(1,-1))
    descript_vec2 = np.reshape(w2,(1,-1))
    #print(descript_vec1)
    #print(descript_vec2)
    return np.concatenate((descript_vec1[0],descript_vec2[0]))

window = np.zeros((2,2))
mw1 = max_pooling(i1_conv1,window)
mw2 = max_pooling(i1_conv2,window)
mw3 = max_pooling(i2_conv1,window)
mw4 = max_pooling(i2_conv2,window)
print("Vectores generados:")
v1 = gen_desc_vect(mw1,mw2)
v2 = gen_desc_vect(mw3,mw4)
print(v1)
print(v2)

#Perceptron

err_vector = [] #vector de error
w_f = [] #vector de pesos finales
#entradas
x = np.array([v1,v2])
print(f"Entradas:\n{x}\n")
#target
class_dict = {0:"A",1:"B"}
t = np.array([[0],[1]])
print(f"Target:\n{t}\n")
for val in t:
    print(class_dict[val[0]])
#aprendizaje
alpha = 0.25
print(f"Alpha:\n{alpha}\n")
bias = np.shape(x)[0] #[0] -> columnas      [1] -> filas
bias = -1*np.ones((bias, 1)) # np.ones((filas,columnas))
print(f"Bias:\n{bias}")
#concatenar x con bias "Vector Aumentado
x = np.concatenate([x, bias], axis = 1) #1 -> añade a la derecha   0 -> añade abajo
#print(f"Matriz Aumentada:\n{x}\n")
#matriz de pesos propuesta
w_i = np.array([[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1]
,[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1]
,[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1]
,[0.1],[0.1],[0.1],[0.1] ])
w_ini = np.copy(w_i)
#w_i = np.random.randn(3, 1) #matriz de pesos aleatorios
print(f"Matriz de pesos propuesta:\n{w_i}\n")
# Regla del perceptrón

# Producto vectorial punto
a = np.dot(x, w_i)
#print(f"a:\n{a}\n")

#Funcion sigmoide
y = 1/(1 + np.exp(-1*a))

#Error cuadratico medio
err = np.sum(0.5 * (t - y)**2)

epoch = 0
epocas = 5000

for i in range (epocas):
    epoch += 1
    for i in range(np.shape(x)[0]): #Repetir segun la cantidad de filas de x
        # Producto punto
        a = np.dot(x[i], w_i)
        #Funcion sigmoide
        y[i] = 1/(1 + np.exp(-1*a))
        # Actualización de pesos
        #print("shape x[i]: 3",np.shape(x[i]))
        x_p = np.reshape(x[i], (len(w_i), 1)) #REACOMODA np.reshape(matriz,(filas,columnas))
        #w_n = w_i + alpha * (t[i] - y[i]) * x_p
        w_n = w_i - alpha * y[i] * (1 - y[i])*(y[i] - t[i]) * x_p
        w_i = w_n
        #print(f"Peso nuevo:\n{w_i}\n")
    # Función de costo -- MSE Error cuadrático medio
    err = (np.sum(0.5 * (t - y)**2)) / len(t)
    err_vector.append(err)
    #print(f"Error:\n{err}\n")
    #Se añade al vector de pesos
    w_f.append(w_i)
    #print(f"Vector de Pesos Finales:\n{w_f}\n")

Vectores generados:
[0. 1. 2. 1. 1. 2. 2. 0. 1. 2. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 0. 0. 1. 1. 0. 0.]
[1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 2. 0. 0.
 1. 2. 2. 0. 0. 1. 2. 1.]
Entradas:
[[0. 1. 2. 1. 1. 2. 2. 0. 1. 2. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1. 1. 0. 0. 1. 1. 0. 0.]
 [1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 2. 0. 0.
  1. 2. 2. 0. 0. 1. 2. 1.]]

Target:
[[0]
 [1]]

A
B
Alpha:
0.25

Bias:
[[-1.]
 [-1.]]
Matriz de pesos propuesta:
[[0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]]



In [2]:
#------------------------------------------------------------------
#Pesos Finales
print('\nPesos finales: ')
for i in range(1):
    res = str(w_i)
    print(res)
    print()

#------------------------------------------------------------------
# Displaying Results
def predict(X):
    if X >= 0.5:
      return 1
    else:
      return 0

a = np.dot(x, w_i)
y = 1/(1 + np.exp(-1*a))

print('PERCEPTRÓN SIGMOIDE')
print('Meta:    Predicción:')
for i in range(len(y)):
    res = str(class_dict[t[i][0]]) + '--------' + str(class_dict[predict(y[i][0])])
    print(res)

print(f'\nÉpocas requeridas: {epoch}.')


Pesos finales: 
[[ 0.36689862]
 [ 0.00857441]
 [-0.61664842]
 [-0.25832421]
 [ 0.00857441]
 [-0.3497498 ]
 [-0.61664842]
 [ 0.1       ]
 [-0.25832421]
 [-0.61664842]
 [ 0.1       ]
 [ 0.1       ]
 [-0.25832421]
 [ 0.1       ]
 [ 0.1       ]
 [ 0.1       ]
 [ 0.36689862]
 [ 0.1       ]
 [ 0.1       ]
 [ 0.1       ]
 [ 0.36689862]
 [ 0.63379725]
 [ 0.1       ]
 [ 0.1       ]
 [ 0.00857441]
 [ 0.27547304]
 [ 0.63379725]
 [ 0.1       ]
 [-0.25832421]
 [ 0.00857441]
 [ 0.63379725]
 [ 0.36689862]
 [ 0.19142559]]

PERCEPTRÓN SIGMOIDE
Meta:    Predicción:
A--------A
B--------B

Épocas requeridas: 5000.


In [3]:
#Se calcula la salida de las imagenes
i1 = np.array([[0,0,0,0,0,0,0],
[0,0,0,0,0,0,0],
[0,0,0,0,0,0,0],
[0,0,0,0,0,1,0],
[0,0,0,0,1,0,0],
[0,0,0,1,0,0,0],
[0,0,0,0,0,0,0]])

i2 = np.array([[0,0,0,0,0,0,0],
[0,0,0,1,0,0,0],
[0,0,0,0,1,0,0],
[0,0,0,0,0,1,0],
[0,0,0,0,0,0,0],
[0,0,0,0,0,0,0],
[0,0,0,0,0,0,0]])

i1_conv1 = convolution(i1,c1)
i1_conv2 = convolution(i1,c2)
i2_conv1 = convolution(i2,c1)
i2_conv2 = convolution(i2,c2)
mw1 = max_pooling(i1_conv1,window)
mw2 = max_pooling(i1_conv2,window)
mw3 = max_pooling(i2_conv1,window)
mw4 = max_pooling(i2_conv2,window)
print("Vectores generados:")
v1 = gen_desc_vect(mw1,mw2)
v2 = gen_desc_vect(mw3,mw4)
print(v1,v2)
x = np.array([v1,v2])
#concatenar x con bias "Vector Aumentado
bias = np.shape(x)[0] #[0] -> columnas      [1] -> filas
bias = -1*np.ones((bias, 1)) # np.ones((filas,columnas))
#print(f"Bias:\n{bias}")
x_aum = np.concatenate([x, bias], axis = 1)
# Producto vectorial punto
a = np.dot(x_aum, w_i)
#print(f"a:\n{a}\n")
#Funcion sigmoide
y = 1/(1 + np.exp(-1*a))
#print(f"y:\n{y}\n")
print('Determinación de clase')
print('Punto:    Predicción:')
for i in range(len(y)):
    res = str(x[i]) + '--------' + str(class_dict[predict(y[i][0])])
    print(res)

Vectores generados:
[0. 0. 0. 1. 0. 0. 1. 2. 0. 1. 2. 2. 0. 1. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 1. 0.] [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 2. 0. 0. 1. 2. 2.
 0. 0. 1. 2. 0. 0. 0. 1.]
Determinación de clase
Punto:    Predicción:
[0. 0. 0. 1. 0. 0. 1. 2. 0. 1. 2. 2. 0. 1. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 1. 0.]--------A
[0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 2. 0. 0. 1. 2. 2.
 0. 0. 1. 2. 0. 0. 0. 1.]--------B
